In [ ]:
# The helpers use the application default credentials.
from cmle_utils import get_models, get_model_versions, train_model, deploy_model, set_default, predict_json, package_and_upload_module

## run locally

### train

In [ ]:

! rm -rf /tmp/cmle
execfile('trainer/task.py')

### predict

In [ ]:
predictions = estimator.predict(input_fn=train_input_fn)

In [ ]:
next(predictions)

## run on CMLE

### train

In [ ]:
# package and upload to gcs before we can train
# this requires having a setup.py in the directory
# this needs to be rerun everytime the trainer model is updated and right before calling train_model
package_path = package_and_upload_module('gs://sandbox-cmle/package')

# this is needed in the job_spec
print(package_path)

In [ ]:
# train in the cloud
import time
import os

BUCKET = ...
JOB_DIR = os.path.join(BUCKET, 'output')

training_inputs = {
    'scaleTier': 'BASIC',
    'packageUris': [package_path],
    'pythonModule': 'trainer.task',
    'region': 'us-central1',
    'jobDir': JOB_DIR,
    'runtimeVersion': '1.8',
    'args': [
        '--steps 1000'
    ]
}
job_spec = {'jobId': 'test_' + str(int(time.time())), 'trainingInput': training_inputs}

In [ ]:
PROJECT_ID = ...
train_model(PROJECT_ID, job_spec)

### deploy


### predict